In [1]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/var/folders/jr/3lphnp956h5_k0h34k1tgkrr0000gn/T/ipykernel_27255/286542778.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/jr/3lphnp956h5_k0h34k1tgkrr0000gn/T/ipykernel_27255/286542778.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
df = spacex_df[spacex_df['Launch Site']=='CCAFS LC-40']
df = df.groupby(['Launch Site','class']).size().reset_index(name='class count')
df

,Launch Site,class,class count
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7


In [5]:
# Create a dash application
app = dash.Dash(__name__)

In [6]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                html.Br(),
                                html.Div(dcc.Dropdown(id='id-site', options=[{'label': 'All Sites', 'value': 'ALL'},
                                                                              {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                                              {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                                              {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                                              {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                                             ],
                                         value='ALL', #default value
                                         placeholder='Select a launch site here',
                                         searchable=True)),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P('Payload range (kg):'),
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]
                                                )),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br(),
                               ])

In [7]:
# add callback decorator
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='id-site', component_property='value'))

# Add computation to callback function and return pie chart
def get_pie_chart(entered_site):
    # Select site
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
                     names='Launch Site',
                     title='Success Count for all launch sites')
        return fig
    else:
        df = spacex_df[spacex_df['Launch Site']==entered_site]
        df = df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(df, values='class count',
                     names='class',
                     title='Success Count for site {}'.format(entered_site))
        return fig

@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='id-site',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])

def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success count on Payload mass for site {entered_site}")
        return fig

In [8]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Apr/2022 19:28:46] "GET /_dash-component-suites/dash/dcc/async-sli